In [9]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

def load_data(file_path):
    data = pd.read_csv(file_path, delimiter=';')
    return data

def filter_data_by_property_type(data, property_type):
    filtered_data = data[data['EJKAT20'] == property_type]
    return filtered_data

def filter_data_by_year(data, years):
    current_year = pd.Timestamp.now().year
    start_year = current_year - years
    filtered_data = data[data['TID'].str[:4].astype(int) >= start_year]
    return filtered_data

def calculate_median_price_change(data):
    price_changes = data['INDHOLD'].values.reshape(-1, 1)
    scaler = StandardScaler()
    scaled_price_changes = scaler.fit_transform(price_changes)
    
    scaled_price_changes_tensor = torch.tensor(scaled_price_changes, dtype=torch.float32)
    
    median_price_change_tensor = torch.median(scaled_price_changes_tensor)
    
    median_price_change = scaler.inverse_transform(median_price_change_tensor.unsqueeze(0).numpy().reshape(1, -1))[0][0]
    return median_price_change

def predict_next_year_change(data):
    data['Year'] = data['TID'].str[:4].astype(int)
    X = data['Year'].values.reshape(-1, 1)
    y = data['INDHOLD'].values
    
    model = LinearRegression()
    model.fit(X, y)
    
    next_year = np.array([[data['Year'].max() + 1]])
    predicted_change = model.predict(next_year)[0]
    return predicted_change

def main():
    file_path = 'C:/Users/Kalle/Documents/GitHub/PriceCalc/EJ121.csv'  
    property_type = 'One-family houses'  
    years = 5  

    data = load_data(file_path)
    data = filter_data_by_property_type(data, property_type)
    data = filter_data_by_year(data, years)
    
    median_price_change = calculate_median_price_change(data)
    predicted_change = predict_next_year_change(data)
    
    print(f"The median price change for {property_type} over the last {years} years is: {median_price_change}%")
    print(f"The predicted percentage change for {property_type} next year is: {predicted_change}%")

if __name__ == "__main__":
    main()

The median price change for One-family houses over the last 5 years is: 1.0%
The predicted percentage change for One-family houses next year is: -0.7013235294118658%
